# Load the Data

In [34]:
# create a list of all the texts
texts = []
with open('testset.txt', 'r') as f:
    for line in f:
        elements = line.strip().split('\t')
        texts.append(elements[0].replace('\n', ''))
        texts.append(elements[1])

In [35]:
for text in texts:
    print(text)

Tasting it is the only reliable way.
The way you have it is fine.
I think it probably depends on your money.
It depends on your country.
You need to read a lot to know what you like and what you don't.
You don't have to know.
Obviously, the best book for you depends a lot on what you are looking for.
The answer will depend of course on what you're looking for in a vacation.
I've had this same problem.
I had the same problem as you.
If you are not sure how to do it, don't do it at all.
Also, if you die, you don't have to repay, don't know if it helps.
It is more deciding what you should do and what you should get others to do.
What can you do and what should you do to grow more?
You need to add this to your resume because it is the only position you have had.
You may want it, but the process given to you is what you have to work within.
I remained under the banyan tree, exhausted by my daily ritual of dragooning the men every two hours. 
I remained under the banyan tree, exhausted by my

# Create Shingles for each text

In [36]:
shingles_per_text = []
k = 4
for text in texts:
    shingles_per_text.append(set([text[i:i + k] for i in range(len(text) - k + 1)]))

print(shingles_per_text[0])

{' rel', 'ting', 'y re', ' onl', 'e wa', 's th', 'ing ', 'stin', 'asti', 'le w', ' way', 't is', 'able', 'is t', 'elia', 'only', 'reli', 'he o', 'Tast', 'ble ', 'liab', ' it ', 'iabl', 'ng i', 'way.', 'ly r', 'it i', 'g it', ' the', 'the ', 'nly ', ' is ', 'e on'}


# Create Bit-Vector for each text (Matrix of Bit-Vectors [f. 51])

In [37]:
# create a set of all shingles
all_shingles_of_file = set()
for shingle_of_text in shingles_per_text:
    all_shingles_of_file.update(shingle_of_text)
print(all_shingles_of_file)

{'ant ', 'xhau', 'babl', 'now.', 've t', ' sam', 'g fo', 'nlin', 'r re', 'prob', ' a l', 'e ho', 'ad t', ' you', 'mone', 'e ca', 'ok f', 'on y', 'king', 't it', 'ntry', ' it,', 'ther', 'how ', 'e an', 've h', 'may ', 'iven', ', ex', 'this', 'lps.', ', yo', 'ur r', 'so, ', 's gi', ' tre', ' ans', 've i', 'on o', 'stin', 'answ', 's ev', 'ore ', 'ecau', 'rs. ', 'd un', 'if y', 'work', 'an t', 'urs.', 'onli', ' a t', 'okin', 'elia', ' if ', 'k fo', 'more', 'w mo', 'know', 'a to', 'here', ' nee', 'ld g', "re's", ' do.', 't pr', 'o ho', 'd to', 'nk i', 'othe', ' eve', 'If y', 'ily ', 'ly, ', "u're", 'yan ', 'sly,', 'aria', 'ends', ' the', 't su', 'ion ', 'ned ', 'ld d', 'r wi', 'ad a', 'ou i', 'tree', 'atio', 'ng t', 'ut t', 'le w', 'ecid', 'sure', 'is s', 'f va', 'a va', 'need', 'ause', 't bo', ' by ', 's yo', 'at c', 'iabl', 're n', 'hat ', ' lik', 'our ', 'al o', 'usly', "e's ", 'thin', 'itua', 'ney.', 'n ev', "you'", 'ay y', 's th', 'asti', 'e no', 'rema', 'nd o', 'on w', "I've", ', do',

In [38]:
# create Bit-Vector for each text -> this will yield the matrix of the bit-vectors
bit_Matrix = []
for shingle_of_text in shingles_per_text:
    bit_vector_for_text = []
    
    for shingle_of_file in all_shingles_of_file:
        
        if shingle_of_file in shingle_of_text:
            bit_vector_for_text.append(1)
        else:
            bit_vector_for_text.append(0)
            
    bit_Matrix.append(bit_vector_for_text)

In [43]:
# nur um zz zeigen, dass es funktioniert
for vector in bit_Matrix:
    print(vector[0:16])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0]
[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0]
[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
